In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/08 22:38:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
!hdfs dfs -cp sf.csv hdfs://nn:9000/sf.csv

In [3]:
!hdfs dfs -cp holidays2.csv hdfs://nn:9000/holidays2.csv

In [4]:
df = (spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .load("hdfs://nn:9000/sf.csv"))

In [5]:
from pyspark.sql.functions import col, expr
cols = [col(c).alias(c.replace(" ", "_")) for c in df.columns]
df.select(cols).write.format("parquet").save("hdfs://nn:9000/sf.parquet")

24/01/08 19:09:10 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
!hdfs dfs -rm hdfs://nn:9000/sf.csv

Deleted hdfs://nn:9000/sf.csv


In [7]:
(spark.read
 .format("parquet")
 .load("hdfs://nn:9000/sf.parquet")
 .createOrReplaceTempView("calls")
)

In [8]:
(spark.read
 .format("csv")
 .option("inferSchema", True)
 .option("header", True)
 .load("hdfs://nn:9000/holidays2.csv")
 .createOrReplaceTempView("holidays")
)

In [9]:
spark.table("calls")

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, Neighborhooods_-_Analysis_Boundaries: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

In [10]:
spark.table("holidays")

DataFrame[date: string, holiday: string]

In [13]:
spark.sql("SHOW TABLES").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|   stinky|      false|
|         |    calls|       true|
|         | holidays|       true|
+---------+---------+-----------+



In [14]:
# SQL
spark.sql("""
SELECT *
FROM calls
INNER JOIN holidays ON calls.Call_Date = holidays.date
LIMIT 5
""").toPandas()

,Call_Number,Unit_ID,Incident_Number,Call_Type,Call_Date,Watch_Date,Received_DtTm,Entry_DtTm,Dispatch_DtTm,Response_DtTm,...,Unit_Type,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhooods_-_Analysis_Boundaries,RowID,case_location,Analysis_Neighborhoods,date,holiday
0,201461772,94,20060706,Medical Incident,05/25/2020,05/25/2020,05/25/2020 01:34:55 PM,05/25/2020 01:34:55 PM,05/25/2020 01:36:14 PM,05/25/2020 01:36:19 PM,...,MEDIC,1,10,10,Bayview Hunters Point,201461772-94,POINT (-122.38626545834089 37.73782577837169),1,05/25/2020,Memorial Day
1,201463874,E32,20060913,Medical Incident,05/25/2020,05/25/2020,05/25/2020 11:16:21 PM,05/25/2020 11:17:54 PM,05/25/2020 11:18:27 PM,05/25/2020 11:19:53 PM,...,ENGINE,1,9,11,Excelsior,201463874-E32,POINT (-122.42955213038633 37.729273863971024),7,05/25/2020,Memorial Day
2,201463019,T18,20060826,Medical Incident,05/25/2020,05/25/2020,05/25/2020 06:39:44 PM,05/25/2020 06:41:34 PM,05/25/2020 06:55:28 PM,05/25/2020 06:57:32 PM,...,TRUCK,2,8,7,Inner Sunset,201463019-T18,POINT (-122.46829612745225 37.751748874988515),14,05/25/2020,Memorial Day
3,201460900,E11,20060611,Medical Incident,05/25/2020,05/25/2020,05/25/2020 09:22:49 AM,05/25/2020 09:24:51 AM,05/25/2020 09:25:17 AM,05/25/2020 09:26:11 AM,...,ENGINE,1,6,8,Mission,201460900-E11,POINT (-122.42413130758577 37.752703146369065),20,05/25/2020,Memorial Day
4,201463227,B03,20060859,Alarms,05/25/2020,05/25/2020,05/25/2020 07:40:19 PM,05/25/2020 07:41:33 PM,05/25/2020 07:41:44 PM,05/25/2020 07:42:33 PM,...,CHIEF,2,3,6,Financial District/South Beach,201463227-B03,POINT (-122.40169229261626 37.78658885907634),8,05/25/2020,Memorial Day


In [15]:
calls = spark.table("calls")
holidays = spark.table("holidays")

In [18]:
calls["Call_Date"] == holidays["date"]

Column<'(Call_Date = date)'>

In [19]:
# default for how is inner join
calls.join(holidays, on=calls["Call_Date"] == holidays["date"]).limit(5).toPandas()

,Call_Number,Unit_ID,Incident_Number,Call_Type,Call_Date,Watch_Date,Received_DtTm,Entry_DtTm,Dispatch_DtTm,Response_DtTm,...,Unit_Type,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhooods_-_Analysis_Boundaries,RowID,case_location,Analysis_Neighborhoods,date,holiday
0,201461772,94,20060706,Medical Incident,05/25/2020,05/25/2020,05/25/2020 01:34:55 PM,05/25/2020 01:34:55 PM,05/25/2020 01:36:14 PM,05/25/2020 01:36:19 PM,...,MEDIC,1,10,10,Bayview Hunters Point,201461772-94,POINT (-122.38626545834089 37.73782577837169),1,05/25/2020,Memorial Day
1,201463874,E32,20060913,Medical Incident,05/25/2020,05/25/2020,05/25/2020 11:16:21 PM,05/25/2020 11:17:54 PM,05/25/2020 11:18:27 PM,05/25/2020 11:19:53 PM,...,ENGINE,1,9,11,Excelsior,201463874-E32,POINT (-122.42955213038633 37.729273863971024),7,05/25/2020,Memorial Day
2,201463019,T18,20060826,Medical Incident,05/25/2020,05/25/2020,05/25/2020 06:39:44 PM,05/25/2020 06:41:34 PM,05/25/2020 06:55:28 PM,05/25/2020 06:57:32 PM,...,TRUCK,2,8,7,Inner Sunset,201463019-T18,POINT (-122.46829612745225 37.751748874988515),14,05/25/2020,Memorial Day
3,201460900,E11,20060611,Medical Incident,05/25/2020,05/25/2020,05/25/2020 09:22:49 AM,05/25/2020 09:24:51 AM,05/25/2020 09:25:17 AM,05/25/2020 09:26:11 AM,...,ENGINE,1,6,8,Mission,201460900-E11,POINT (-122.42413130758577 37.752703146369065),20,05/25/2020,Memorial Day
4,201463227,B03,20060859,Alarms,05/25/2020,05/25/2020,05/25/2020 07:40:19 PM,05/25/2020 07:41:33 PM,05/25/2020 07:41:44 PM,05/25/2020 07:42:33 PM,...,CHIEF,2,3,6,Financial District/South Beach,201463227-B03,POINT (-122.40169229261626 37.78658885907634),8,05/25/2020,Memorial Day


In [21]:
(calls
 .join(holidays, on=calls["Call_Date"] == holidays["date"], how="inner")
 .limit(5)
 .toPandas()
)

,Call_Number,Unit_ID,Incident_Number,Call_Type,Call_Date,Watch_Date,Received_DtTm,Entry_DtTm,Dispatch_DtTm,Response_DtTm,...,Unit_Type,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhooods_-_Analysis_Boundaries,RowID,case_location,Analysis_Neighborhoods,date,holiday
0,201461772,94,20060706,Medical Incident,05/25/2020,05/25/2020,05/25/2020 01:34:55 PM,05/25/2020 01:34:55 PM,05/25/2020 01:36:14 PM,05/25/2020 01:36:19 PM,...,MEDIC,1,10,10,Bayview Hunters Point,201461772-94,POINT (-122.38626545834089 37.73782577837169),1,05/25/2020,Memorial Day
1,201463874,E32,20060913,Medical Incident,05/25/2020,05/25/2020,05/25/2020 11:16:21 PM,05/25/2020 11:17:54 PM,05/25/2020 11:18:27 PM,05/25/2020 11:19:53 PM,...,ENGINE,1,9,11,Excelsior,201463874-E32,POINT (-122.42955213038633 37.729273863971024),7,05/25/2020,Memorial Day
2,201463019,T18,20060826,Medical Incident,05/25/2020,05/25/2020,05/25/2020 06:39:44 PM,05/25/2020 06:41:34 PM,05/25/2020 06:55:28 PM,05/25/2020 06:57:32 PM,...,TRUCK,2,8,7,Inner Sunset,201463019-T18,POINT (-122.46829612745225 37.751748874988515),14,05/25/2020,Memorial Day
3,201460900,E11,20060611,Medical Incident,05/25/2020,05/25/2020,05/25/2020 09:22:49 AM,05/25/2020 09:24:51 AM,05/25/2020 09:25:17 AM,05/25/2020 09:26:11 AM,...,ENGINE,1,6,8,Mission,201460900-E11,POINT (-122.42413130758577 37.752703146369065),20,05/25/2020,Memorial Day
4,201463227,B03,20060859,Alarms,05/25/2020,05/25/2020,05/25/2020 07:40:19 PM,05/25/2020 07:41:33 PM,05/25/2020 07:41:44 PM,05/25/2020 07:42:33 PM,...,CHIEF,2,3,6,Financial District/South Beach,201463227-B03,POINT (-122.40169229261626 37.78658885907634),8,05/25/2020,Memorial Day


In [23]:
# how many calls on each kind of holiday?
(calls
 .join(holidays, on=calls["Call_Date"] == holidays["date"], how="inner")
 .groupby("holiday")
 .count()
 .orderBy("count")
 .toPandas()
)

,holiday,count
0,Inauguration Day,798
1,Juneteenth National Independence Day,1968
2,Christmas Day,8179
3,Thanksgiving Day,8186
4,Veterans Day,8674
5,Memorial Day,8830
6,Washington's Birthday,9064
7,"Birthday of Martin Luther King, Jr.",9557
8,Labor Day,9778
9,New Year's Day,9966


In [24]:
from pyspark.sql.functions import expr, col

In [25]:
# what perc of fire dept calls on holidays?
(calls
 .join(holidays, on=calls["Call_Date"] == holidays["date"], how="left")
 .agg(expr("COUNT(holiday) / COUNT(*)"))
 .toPandas()
)

,(COUNT(holiday) / COUNT(1))
0,0.015863


# Caching

In [28]:
import requests

# https://spark.apache.org/docs/latest/monitoring.html#rest-api
# http://localhost:4040/api/v1/applications
# http://localhost:4040/api/v1/applications/{app_id}/executors
# look for "totalTasks"

r = requests.get("http://localhost:4040/api/v1/applications")
r.raise_for_status()
r.json()

[{'id': 'app-20240108190848-0000',
  'name': 'cs544',
  'attempts': [{'startTime': '2024-01-08T19:08:47.633GMT',
    'endTime': '1969-12-31T23:59:59.999GMT',
    'lastUpdated': '2024-01-08T19:08:47.633GMT',
    'duration': 2410827,
    'sparkUser': 'root',
    'completed': False,
    'appSparkVersion': '3.5.0',
    'startTimeEpoch': 1704740927633,
    'endTimeEpoch': -1,
    'lastUpdatedEpoch': 1704740927633}]}]

In [29]:
app_id = r.json()[0]["id"]
app_id

'app-20240108190848-0000'

In [30]:
r = requests.get(f"http://localhost:4040/api/v1/applications/{app_id}/executors")
r.raise_for_status()
[exec["totalTasks"] for exec in r.json()]

[0, 39, 33]

In [31]:
df = spark.table("calls").sample(True, 0.01).repartition(1).cache()

In [32]:
df.count()

60257

In [33]:
r = requests.get(f"http://localhost:4040/api/v1/applications/{app_id}/executors")
r.raise_for_status()
[exec["totalTasks"] for exec in r.json()]

[0, 44, 36]

In [34]:
for i in range(30):
    df.count()

In [35]:
r = requests.get(f"http://localhost:4040/api/v1/applications/{app_id}/executors")
r.raise_for_status()
[exec["totalTasks"] for exec in r.json()]

[0, 74, 36]

### cache with MEMORY_2, then they both would have had some work

In [43]:
import pyspark

In [45]:
df = (spark.table("calls").sample(True, 0.01).repartition(1)
      .persist(pyspark.StorageLevel.MEMORY_ONLY_2)
     )
df.count()

60265

In [46]:
r = requests.get(f"http://localhost:4040/api/v1/applications/{app_id}/executors")
r.raise_for_status()
[exec["totalTasks"] for exec in r.json()]

[0, 77, 41]

In [47]:
for i in range(30):
    df.count()

In [48]:
r = requests.get(f"http://localhost:4040/api/v1/applications/{app_id}/executors")
r.raise_for_status()
[exec["totalTasks"] for exec in r.json()]

[0, 92, 56]

# Concept of Hash Partitioning

In [49]:
hash(b"a")

-291891326222623123

In [50]:
hash(b"b")


-3202341361808504782

In [52]:
 spark.sql("""
SELECT Call_Type, COUNT(*) as count
FROM calls
GROUP BY Call_Type
""").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Call_Type#231], functions=[count(1)])
   +- Exchange hashpartitioning(Call_Type#231, 200), ENSURE_REQUIREMENTS, [plan_id=2462]
      +- HashAggregate(keys=[Call_Type#231], functions=[partial_count(1)])
         +- FileScan parquet [Call_Type#231] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[hdfs://nn:9000/sf.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Call_Type:string>




In [51]:
 spark.sql("""
SELECT Call_Type, COUNT(*) as count
FROM calls
GROUP BY Call_Type
""").explain("formatted")

== Physical Plan ==
AdaptiveSparkPlan (5)
+- HashAggregate (4)
   +- Exchange (3)
      +- HashAggregate (2)
         +- Scan parquet  (1)


(1) Scan parquet 
Output [1]: [Call_Type#231]
Batched: true
Location: InMemoryFileIndex [hdfs://nn:9000/sf.parquet]
ReadSchema: struct<Call_Type:string>

(2) HashAggregate
Input [1]: [Call_Type#231]
Keys [1]: [Call_Type#231]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#48157L]
Results [2]: [Call_Type#231, count#48158L]

(3) Exchange
Input [2]: [Call_Type#231, count#48158L]
Arguments: hashpartitioning(Call_Type#231, 200), ENSURE_REQUIREMENTS, [plan_id=2449]

(4) HashAggregate
Input [2]: [Call_Type#231, count#48158L]
Keys [1]: [Call_Type#231]
Functions [1]: [count(1)]
Aggregate Attributes [1]: [count(1)#48154L]
Results [2]: [Call_Type#231, count(1)#48154L AS count#48153L]

(5) AdaptiveSparkPlan
Output [2]: [Call_Type#231, count#48153L]
Arguments: isFinalPlan=false




In [53]:
# spark.table("calls").write.bucketBy(10, "Call_Type").saveAsTable("call_by_type")

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, Neighborhooods_-_Analysis_Boundaries: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

In [54]:
# sample is just to make the demo faster, it would work without that
(spark.table("calls")
 .sample(True, 0.01)
 # .repartition(10, "Call_Type")
 .write
 .bucketBy(10, "Call_Type")
 # .mode("overwrite")
 .saveAsTable("calls_by_type"))

24/01/08 20:13:27 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/01/08 20:13:27 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/01/08 20:13:27 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/01/08 20:13:27 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [56]:
spark.sql("""
SELECT Call_Type, COUNT(*) as count
FROM calls_by_type
GROUP BY Call_Type
""").explain("formatted")

== Physical Plan ==
AdaptiveSparkPlan (4)
+- HashAggregate (3)
   +- HashAggregate (2)
      +- Scan parquet spark_catalog.default.calls_by_type (1)


(1) Scan parquet spark_catalog.default.calls_by_type
Output [1]: [Call_Type#48281]
Batched: true
Bucketed: true
Location: InMemoryFileIndex [hdfs://nn:9000/user/hive/warehouse/calls_by_type]
ReadSchema: struct<Call_Type:string>
SelectedBucketsCount: 10 out of 10

(2) HashAggregate
Input [1]: [Call_Type#48281]
Keys [1]: [Call_Type#48281]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#48316L]
Results [2]: [Call_Type#48281, count#48317L]

(3) HashAggregate
Input [2]: [Call_Type#48281, count#48317L]
Keys [1]: [Call_Type#48281]
Functions [1]: [count(1)]
Aggregate Attributes [1]: [count(1)#48277L]
Results [2]: [Call_Type#48281, count(1)#48277L AS count#48276L]

(4) AdaptiveSparkPlan
Output [2]: [Call_Type#48281, count#48276L]
Arguments: isFinalPlan=false




# JOIN algorithms

In [57]:
# kind_id, color
fruits = [
    ("B", "Yellow"),
    ("A", "Green"),
    ("C", "Orange"),
    ("A", "Red"),
    ("C", "Purple"),
    ("B", "Green")
]

# kind_id, name (assume no duplicate kind_id's)
kinds = [
    ("A", "Apple"),
    ("B", "Banana"),
    ("C", "Carrot")
]

# GOAL: print Yellow Banana, Green Apple, etc (any order)

### hash join

In [ ]:
kind_lookup = dict(kinds)
kind_lookup

In [59]:
for kind_id, color in fruits:
    print(color, kind_lookup[kind_id])

Yellow Banana
Green Apple
Orange Carrot
Red Apple
Purple Carrot
Green Banana


### sort merge join

In [ ]:
fruits.sort()
kinds.sort()
fruits

In [66]:
fruit_idx = 0
for kind_id, food_name in kinds:
    # print(food_name)
    while fruit_idx < len(fruits):
        if fruits[fruit_idx][0] > kind_id: 
            break
        elif fruits[fruit_idx][0] == kind_id:
            print(food_name, fruits[fruit_idx][1])
            
        fruit_idx += 1

Apple Green
Apple Red
Banana Green
Banana Yellow
Carrot Orange
Carrot Purple


### BHJ: Broadcast Hash Join

### SMJ: Shuffle Sort Merge Join